In [5]:
import aioredis
import json
redis_endpoint = aioredis.from_url("redis://localhost")

keys = await redis_endpoint.keys()
data_array = []

for key in keys:
  value = await redis_endpoint.get(key)
  data = json.loads(value)
  data_array.append(data)

In [4]:
file_path = "/home/semantic-server/soganghci/api-server/scripts/script.json"
json_string = json.dumps(data_array, indent=4)

with open(file_path, "w") as f:
    f.write(json_string)

In [34]:
filtered_array = [item for item in data_array if "init-timestamp" in item and "dialogue_korean" in item]
print(filtered_array)
sorted_array = sorted(filtered_array, key=lambda x: x.get("init-timestamp", float('inf')))
print(sorted_array)

[{'init': '07-30 10:20:10', 'init-timestamp': 1690680010.305434, 'dialogue_korean': [{'time': 1690680012.311359, 'role': 'user', 'content': 'Hello'}, {'time': 1690680013.682628, 'role': 'assistant', 'content': '환영합니다, 저는 파블로 피카소라고 하는 스페인의 화가입니다. 당신에 대해 소개해줄 수 있나요?'}, {'time': 1690680054.739153, 'role': 'user', 'content': '안녕하세요. 저는...'}, {'time': 1690680056.559778, 'role': 'assistant', 'content': '그렇군요, 여기서 만나게 되서 정말 기뻐요. 어떻게 이곳에 오게 되었나요?'}, {'time': 1690680074.652753, 'role': 'user', 'content': '전시회를 보러 왔습니다.'}, {'time': 1690680076.574913, 'role': 'assistant', 'content': '당신에게 저의 그림 게르니카를 소개할 수 있어서 정말 기쁩니다. 함께 저와 그림을 감상하시겠어요?'}, {'time': 1690680092.652009, 'role': 'user', 'content': '네.'}, {'time': 1690680094.164501, 'role': 'assistant', 'content': '그림에서 무슨 일이 일어나고 있나요?'}, {'time': 1690680109.27118, 'role': 'user', 'content': '전쟁이 일어나고 있는 것 같습니다.'}, {'time': 1690680110.98907, 'role': 'assistant', 'content': '무엇을 보고 그렇게 말하셨나요?'}, {'time': 1690680121.409196, 'role': 'user', 'content': '

In [36]:
import pandas as pd

def flatten_dialogues(data):
    init = data["init"]
    init_timestamp = data["init-timestamp"]
    dialogue_korean = data.get("dialogue_korean", [])
    print(dialogue_korean)
    dialogue = data.get("dialogue", [])

    rows = []
    for d in dialogue:
        time = d.get("time", None)
        role = d.get("role", None)
        content = d.get("content", None)
        rows.append([time, role, content])

    # If there are any remaining dialogue_korean elements, append them to rows
    for korean_data in dialogue_korean:
        time = korean_data.get("time", None)
        role = korean_data.get("role", None)
        content = korean_data.get("content", None)
        # korean_time = korean_data.get("time", None)
        # korean_role = korean_data.get("role", None)
        # korean_content = korean_data.get("content", None)
        # rows.append([None, None, None, korean_time, korean_role, korean_content])
        rows.append([time, role, content])

    # Create DataFrame and convert it to CSV
    # df = pd.DataFrame(rows, columns=["time", "role", "content", "dialogue_korean_time", "dialogue_korean_role", "dialogue_korean_content"])
    df = pd.DataFrame(rows, columns=["time", "role", "content"])

    return df

for item in sorted_array:
  df = flatten_dialogues(item)
  df.to_csv(f"{item['init']}.csv", index=False)

[{'role': 'user', 'content': 'Hello'}, {'role': 'assistant', 'content': '환영합니다, 저는 파블로 피카소라고 하는 스페인의 화가입니다. 당신에 대해 소개해줄 수 있나요?'}, {'role': 'user', 'content': 'My name is Jeong Daeheon.'}, {'role': 'assistant', 'content': '그렇군요, 여기서 만나게 되서 정말 기뻐요. 어떻게 이곳에 오게 되었나요?'}, {'role': 'user', 'content': 'I received an invitation from people.'}, {'role': 'assistant', 'content': '당신에게 저의 그림 게르니카를 소개할 수 있어서 정말 기쁩니다. 함께 저와 그림을 감상하시겠어요?'}, {'role': 'user', 'content': 'I like it.'}, {'role': 'assistant', 'content': '그림에서 무슨 일이 일어나고 있나요?'}, {'role': 'user', 'content': 'I am screaming.'}, {'role': 'assistant', 'content': '무엇을 보고 그렇게 말하셨나요?'}, {'role': 'user', 'content': 'Human facial expressions'}, {'role': 'assistant', 'content': '죄송하지만 이만 가보겠습니다. 하지만 게르니카의 깊이를 직접 탐험해 보시길 권합니다. 게르니카는 깊은 감정을 불러일으키고 의미 있는 토론을 불러일으키는 힘이 있습니다. 예술 탐험을 즐기세요!'}]
[{'role': 'user', 'content': 'Hello'}, {'role': 'assistant', 'content': '환영합니다, 저는 파블로 피카소라고 하는 스페인의 화가입니다. 당신에 대해 소개해줄 수 있나요?'}, {'role': 'user', 'content': 'My name

In [1]:
import os
import pandas as pd

def process_csv_files(input_directory, output_directory):
    # Get a list of all files in the input directory
    file_list = [f for f in os.listdir(input_directory) if f.endswith(".csv")]

    # Loop through each file
    for file_name in file_list:
        input_path = os.path.join(input_directory, file_name)
        output_path = os.path.join(output_directory, file_name)

        # Load the CSV file using pandas
        df = pd.read_csv(input_path)

        # Do any processing or manipulation with the data here
        # For example, you can add columns, filter data, etc.

        # Save the processed data as a new CSV file in the output directory
        df.to_csv(output_path, index=False, encoding="utf-8-sig")

        print(f"Processed CSV file '{file_name}' and saved to '{output_path}'")

# Example usage:
input_dir = "/home/semantic-server/soganghci/api-server/scripts/excel"
output_dir = "/home/semantic-server/soganghci/api-server/scripts/excel_new"

process_csv_files(input_dir, output_dir)

Processed CSV file '07-30 14:09:22 id11.csv' and saved to '/home/semantic-server/soganghci/api-server/scripts/excel_new/07-30 14:09:22 id11.csv'
Processed CSV file '07-30 16:05:38 id18.csv' and saved to '/home/semantic-server/soganghci/api-server/scripts/excel_new/07-30 16:05:38 id18.csv'
Processed CSV file '07-30 14:45:58 id13.csv' and saved to '/home/semantic-server/soganghci/api-server/scripts/excel_new/07-30 14:45:58 id13.csv'
Processed CSV file '07-30 15:00:37 id14.csv' and saved to '/home/semantic-server/soganghci/api-server/scripts/excel_new/07-30 15:00:37 id14.csv'
Processed CSV file '07-30 11:35:22 id6.csv' and saved to '/home/semantic-server/soganghci/api-server/scripts/excel_new/07-30 11:35:22 id6.csv'
Processed CSV file '07-30 15:16:04 id15.csv' and saved to '/home/semantic-server/soganghci/api-server/scripts/excel_new/07-30 15:16:04 id15.csv'
Processed CSV file '07-30 13:25:48 id9.csv' and saved to '/home/semantic-server/soganghci/api-server/scripts/excel_new/07-30 13:25:4